In [1]:
from OptiHPLCHandler import EmpowerHandler
from pprint import pprint
import sys

sys.path.insert(0, "../")

In [2]:
# get api address from .env file
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore") # Required to suppress warnings from requests 

load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

Get instrument method

In [3]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout = 300
handler.connection.default_post_timeout = 300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "20240430_slcb"
    full_method = handler.GetInstrumentMethod(method_name)

Original method name and gradient table

In [4]:
print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

original_method_name = full_method.method_name

20240430_slcb
-------------
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.00'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.10'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.10'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.20'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.00'}]


Generate ramp-up method

In [5]:
from applications.method_generators.ramp_method import generate_ramp_method
from applications.revert_method import revert_method

generate_ramp_method(
    method=full_method,  # basis instrument method
    ramp_time=10,  # ramp time in minutes
    low_flow_rate=0.03,  # low flow rate in ml/min
    ramp_type="rampup",  # rampup or rampdown
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

20240430_slcb_ramp
------------------
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.03',
  'Time': 'Initial'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10'}]


Revert method to original method to generate other methods

In [6]:
revert_method(method=full_method, original_method_name=original_method_name)
print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

20240430_slcb
-------------
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.00'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.10'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.10'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.20'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.00'}]


In [7]:
from applications.method_generators.condense_gradient_table import (
    generate_condense_gradient_table,
)

generate_condense_gradient_table(
    method=full_method,  # basis instrument method
    new_method_time=10,  # time to condense gradient to
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

20240430_slcb_cond_10m
----------------------
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '5.0'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '5.05'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '6.05'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '6.1'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.0'}]


In [9]:
from applications.method_generators.alter_temperature import (
    generate_altered_temperature_method,
)

generate_altered_temperature_method(
    method=full_method,  # basis instrument method
    temperature_delta=5,  # adds temperature delta to current temperature, accepts negative values
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.column_temperature)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

20240430_slcb_5_0C
------------------
'50.0'


In [10]:
from applications.method_generators.alter_strong_eluent_pct import (
    generate_altered_strong_eluent_method_pct,
)

generate_altered_strong_eluent_method_pct(
    method=full_method,  # basis instrument method
    strong_eluent_delta=1,  # adds strong eluent delta, accepts negative values
    maintain_wash_pct=True,  # false if the end of the gradient is the wash step
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

20240430_slcb_1_0pct
--------------------
[{'CompositionA': '69.0',
  'CompositionB': '31.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '39.0',
  'CompositionB': '61.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.00'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.10'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.10'},
 {'CompositionA': '69.0',
  'CompositionB': '31.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '12.20'},
 {'CompositionA': '69.0',
  'CompositionB': '31.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.00'}]


In [11]:
from applications.method_generators.add_isocratic_segment import (
    generate_add_isocratic_segment_to_method,
)

generate_add_isocratic_segment_to_method(
    method=full_method,  # basis instrument method
    isocratic_duration=10,  # duration of isocratic step
    index_of_isocratic_segment=0,  # where isocratic step is added, 0 for beginning, -1 for end
)

print(full_method.method_name)
print("-" * len(full_method.method_name))
pprint(full_method.gradient_table)

# revert
revert_method(method=full_method, original_method_name=original_method_name)

20240430_slcb_iso_10m_0
-----------------------
[{'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': 'Initial',
  'Flow': '0.300',
  'Time': 'Initial'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '10.0'},
 {'CompositionA': '40.0',
  'CompositionB': '60.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.0'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '20.1'},
 {'CompositionA': '10.0',
  'CompositionB': '90.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '22.1'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '22.2'},
 {'CompositionA': '70.0',
  'CompositionB': '30.0',
  'Curve': '6',
  'Flow': '0.300',
  'Time': '30.0'}]
